<a href="https://colab.research.google.com/github/shivaavvari/EDA/blob/main/CreditcardFraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlxtend
!pip intall imblearn

import kagglehub
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics  import confusion_matrix ,roc_auc_score ,roc_curve, ConfusionMatrixDisplay ,RocCurveDisplay,f1_score,precision_score,recall_score,accuracy_score
import pickle
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

import seaborn as sns
%matplotlib inline


# Download latest version
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

In [ ]:
df = pd.read_csv("/root/.cache/kagglehub/datasets/mlg-ulb/creditcardfraud/versions/3/creditcard.csv",sep=",")

In [ ]:
df.isna().sum()
df.info()
df.dtypes
df.describe()


In [ ]:
fig,ax = plt.subplots(nrows=6,ncols=5,figsize=(20,10))
for col in df.columns[:-1]:
  plt.subplot(6,5,df.columns.get_loc(col)+1)
  df[col].plot(kind='hist')
  plt.tight_layout()
plt.show()


In [ ]:
sns.pairplot(df[df.columns[:-1]])

In [ ]:
sns.heatmap(df.corr())

In [ ]:
df.corr().plot(legend=False)

# New Section

In [ ]:
X =df[df.columns[:-1]]
y = df[df.columns[-1]]

In [ ]:
scaler  = StandardScaler()
scaler.fit_transform(X)
X_scaled = scaler.transform(X)


In [ ]:

X_train,X_test,y_train,y_test = train_test_split(X_scaled, y,test_size=0.33,random_state=42)

In [ ]:
df.columns


In [ ]:

cls = RandomForestClassifier()

cls.fit(X_train,y_train )

In [ ]:
y_pred = cls.predict(X_test)

test = X_test[y_pred==1]
type(test)

In [ ]:
X_test [y_pred == 1]
df_test = pd.DataFrame(X_test[y_pred==1],columns=df.columns[:-1])
df_test

In [ ]:
df_test[df_test.index==1].to_json().replace('"0":','').replace("{",'').replace("}","")

In [ ]:
cm = confusion_matrix(y_test,y_pred)
cm_display =ConfusionMatrixDisplay(cm,display_labels=[0,1])
cm_display.plot()
plt.show()

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
f1_score(y_test,y_pred)

In [ ]:
precision_score(y_test,y_pred)

In [ ]:
recall_score(y_test,y_pred)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
y_score = cls.predict_proba(X_test)
y_score =y_score[:,1]
roc_auc_score(y_test,y_score)


In [ ]:

fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc_score(y_test,y_score)
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr,tpr,'b',label="auc="+str(roc_auc_score(y_test,y_score)))
plt.legend(loc=4)
plt.show()

In [ ]:
PrecisionRecallDisplay.from_estimator(cls,X_test,y_test,plot_chance_level=True)

In [ ]:
PrecisionRecallDisplay.from_predictions(y_test,y_pred)

In [ ]:
import pickle
with open("model_rf.pkl",'wb') as f:
  pickle.dump(cls,f)
  f.close()

In [ ]:
weak_learner = DecisionTreeClassifier(max_leaf_nodes=8)

In [ ]:
clf = AdaBoostClassifier(estimator=weak_learner,n_estimators=200,algorithm='SAMME',random_state=42)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
y_pred_ada = clf.predict(X_test)

In [ ]:
cm_ada=confusion_matrix(y_test,y_pred_ada)

In [ ]:
cm_ada_display =ConfusionMatrixDisplay(cm_ada,display_labels=[0,1])
cm_ada_display.plot()
plt.show()

In [ ]:
print(classification_report(y_test,y_pred_ada))

In [ ]:
f1_score(y_test,y_pred_ada)

In [ ]:
precision_score(y_test,y_pred_ada)


In [ ]:
accuracy_score(y_test,y_pred_ada )

In [ ]:
PrecisionRecallDisplay.from_estimator(clf,X_test,y_test,plot_chance_level=True)

In [ ]:
PrecisionRecallDisplay.from_predictions(y_test,y_pred_ada)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
y_score_ada = clf.predict_proba(X_train)[:,1]
roc_auc_score(y_train,y_score_ada)

In [ ]:
with open("model_ada.pkl",'wb') as f:
  pickle.dump(clf,f)
  f.close()

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train,y_train)

In [ ]:
y_pred = lr.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
cm_display =ConfusionMatrixDisplay(cm,display_labels=[0,1])
cm_display.plot()
plt.show()

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
f1_score(y_test,y_pred)
precision_score(y_test,y_pred)
recall_score(y_test,y_pred)
accuracy_score(y_test,y_pred)
y_score = lr.predict_proba(X_test)
y_score =y_score[:,1]
roc_auc_score(y_test,y_score)


In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc_score(y_test,y_score)
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr,tpr,'b',label="auc="+str(roc_auc_score(y_test,y_score)))
plt.legend(loc=4)
plt.show()

In [ ]:
import pickle
with open("model_lr.pkl",'wb') as f:
  pickle.dump(lr,f)
  f.close()

Support Vector Machines

In [ ]:
from sklearn.svm import SVC
svm = SVC(probability=True)
svm.fit(X_train,y_train)

In [ ]:
y_pred = lr.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
cm_display =ConfusionMatrixDisplay(cm,display_labels=[0,1])
cm_display.plot()
plt.show()

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
f1_score(y_test,y_pred)
precision_score(y_test,y_pred)
recall_score(y_test,y_pred)
accuracy_score(y_test,y_pred)
y_score = svm.predict_proba(X_test)
y_score =y_score[:,1]
roc_auc_score(y_test,y_score)


In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc_score(y_test,y_score)
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr,tpr,'b',label="auc="+str(roc_auc_score(y_test,y_score)))
plt.legend(loc=4)
plt.show()

In [ ]:
with open("model_svm.pkl",'wb') as f:
  pickle.dump(svm,f)
  f.close()

K nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)

In [ ]:
y_pred = knn.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
cm_display =ConfusionMatrixDisplay(cm,display_labels=[0,1])
cm_display.plot()
plt.show()

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
f1_score(y_test,y_pred)
precision_score(y_test,y_pred)
recall_score(y_test,y_pred)
accuracy_score(y_test,y_pred)
y_score = Knn.predict_proba(X_test)
y_score =y_score[:,1]
roc_auc_score(y_test,y_score)


In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc_score(y_test,y_score)
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr,tpr,'b',label="auc="+str(roc_auc_score(y_test,y_score)))
plt.legend(loc=4)
plt.show()

In [ ]:
with open("model_knn.pkl",'wb') as f:
  pickle.dump(knn,f)
  f.close()


NaiveBayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train,y_train)


In [ ]:
y_pred = nb.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
cm_display =ConfusionMatrixDisplay(cm,display_labels=[0,1])
cm_display.plot()
plt.show()

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
f1_score(y_test,y_pred)
precision_score(y_test,y_pred)
recall_score(y_test,y_pred)
accuracy_score(y_test,y_pred)
y_score = nb.predict_proba(X_test)
y_score =y_score[:,1]
roc_auc_score(y_test,y_score)


In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc_score(y_test,y_score)
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr,tpr,'b',label="auc="+str(roc_auc_score(y_test,y_score)))
plt.legend(loc=4)
plt.show()

In [ ]:
with open("model_nb.pkl",'wb') as f:
  pickle.dump(nb,f)
  f.close()

Decision Tree classifier


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

In [ ]:
y_pred = dt.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
cm_display =ConfusionMatrixDisplay(cm,display_labels=[0,1])
cm_display.plot()
plt.show()

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
f1_score(y_test,y_pred)
precision_score(y_test,y_pred)
recall_score(y_test,y_pred)
accuracy_score(y_test,y_pred)
y_score = dt.predict_proba(X_test)
y_score =y_score[:,1]
roc_auc_score(y_test,y_score)

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc_score(y_test,y_score)
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr,tpr,'b',label="auc="+str(roc_auc_score(y_test,y_score)))
plt.legend(loc=4)
plt.show()

In [ ]:
with open("model_dt.pkl",'wb') as f:
  pickle.dump(dt,f)
  f.close()


MLPClassifier: Multilayer Perceptron classifier.

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(X_train,y_train)

In [ ]:
y_pred = mlp.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
cm_display =ConfusionMatrixDisplay(cm,display_labels=[0,1])
cm_display.plot()
plt.show()

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
f1_score(y_test,y_pred)
precision_score(y_test,y_pred)
recall_score(y_test,y_pred)
accuracy_score(y_test,y_pred)
y_score = mlp.predict_proba(X_test)
y_score =y_score[:,1]
roc_auc_score(y_test,y_score)

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc_score(y_test,y_score)
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr,tpr,'b',label="auc="+str(roc_auc_score(y_test,y_score)))
plt.legend(loc=4)
plt.show()

In [ ]:
with open("model_mpl.pkl",'wb') as f:
  pickle.dump(mpl,f)
  f.close()

Synthetic Minority oversampling method -lr

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_smote,y_smote = smote.fit_resample(X,y)
X_train_smote,X_test_smote,y_train_smote,y_test_smote = train_test_split(X_smote, y_smote,test_size=0.33,random_state=42)



In [ ]:
with open("model_lr.pkl",'rb') as f:
  lr = pickle.load(f)
  f.close()

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000,solver='liblinear')
lr.fit(X_train_smote,y_train_smote)
y_pred_smote = lr.predict(X_test_smote)

In [ ]:
print(classification_report(y_test_smote,y_pred_smote))

In [ ]:
f1_score(y_test_smote,y_pred_smote)
precision_score(y_test_smote,y_pred_smote)
recall_score(y_test_smote,y_pred_smote)
accuracy_score(y_test_smote,y_pred_smote)
y_score = lr.predict_proba(X_test_smote)
y_score =y_score[:,1]
roc_auc_score(y_test_smote,y_score)


In [ ]:
fpr, tpr, _ = roc_curve(y_test_smote, y_score)
roc_auc_score(y_test_smote,y_score)
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr,tpr,'b',label="auc="+str(roc_auc_score(y_test_smote,y_score)))
plt.legend(loc=4)
plt.show()

In [ ]:
with open("model_svm.pkl",'rb') as f:
  svm = pickle.load(f)
  f.close()

Synthetic Minority oversampling method - SVM

In [ ]:
from sklearn.svm import LinearSVC
svm = LinearSVC()
svm.fit(X_train_smote,y_train_smote)
y_pred_smote = svm.predict(X_test_smote)

In [ ]:
print(classification_report(y_test_smote,y_pred_smote))

In [ ]:
f1_score(y_test_smote,y_pred_smote)
precision_score(y_test_smote,y_pred_smote)
recall_score(y_test_smote,y_pred_smote)
accuracy_score(y_test_smote,y_pred_smote)
y_pred_smote = svm.predict(X_test_smote)


In [ ]:
fpr, tpr, _ = roc_curve(y_test_smote, y_pred))
roc_auc_score(y_test_smote,y_score)
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr,tpr,'b',label="auc="+str(roc_auc_score(y_test_smote,y_score)))
plt.legend(loc=4)
plt.show()

Synthetic Minority oversampling method - KNN






In [ ]:
knn.fit(X_train_smote,y_train_smote)
y_pred_smote = knn.predict(X_test_smote)

In [ ]:
print(classification_report(y_test_smote,y_pred_smote))

In [ ]:
f1_score(y_test_smote,y_pred_smote)
precision_score(y_test_smote,y_pred_smote)
recall_score(y_test_smote,y_pred_smote)
accuracy_score(y_test_smote,y_pred_smote)
y_score = svm.predict_proba(X_test_smote)
y_score =y_score[:,1]
roc_auc_score(y_test_smote,y_score)

In [ ]:
fpr, tpr, _ = roc_curve(y_test_smote, y_score)
roc_auc_score(y_test_smote,y_score)
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr,tpr,'b',label="auc="+str(roc_auc_score(y_test_smote,y_score)))
plt.legend(loc=4)
plt.show()

Synthetic Minority oversampling method - NB


In [ ]:
nb.fit(X_train_smote,y_train_smote)
y_pred_smote = nb.predict(X_test_smote)

In [ ]:
print(classification_report(y_test_smote,y_pred_smote))

In [ ]:
f1_score(y_test_smote,y_pred_smote)
precision_score(y_test_smote,y_pred_smote)
recall_score(y_test_smote,y_pred_smote)
accuracy_score(y_test_smote,y_pred_smote)
y_score = nb.predict_proba(X_test_smote)
y_score =y_score[:,1]
roc_auc_score(y_test_smote,y_score)

In [ ]:
fpr, tpr, _ = roc_curve(y_test_smote, y_score)
roc_auc_score(y_test_smote,y_score)
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr,tpr,'b',label="auc="+str(roc_auc_score(y_test_smote,y_score)))
plt.legend(loc=4)
plt.show()